In [ ]:
# This notebook illustrates password case 
# unpack prepared combinatorics dataset from passwords.zip in the directory containing this
# notebook

import os, shutil
import numpy as np
import keras
from keras.models import Sequential

partition = {}
labels = {}

goods = []
bads = []

for line in open('goodpasswords.txt'):
    word =line.strip().split(',')[0]
    goods.append('psw'+word+'.npz')
    
for line in open('badpasswords.txt'):
    word =line.strip().split(',')[0]
    bads.append('psw'+word+'.npz')


base_dir = './passwords'
try:
    os.mkdir(base_dir)
except:
    pass

train_dir = os.path.join(base_dir,'train')

try:
    os.mkdir(train_dir)
except:
    pass

train_goods_dir = os.path.join(train_dir, 'goods')
try:
    os.mkdir(train_goods_dir)
except:
    pass


partition['train'] = goods[:1000]


for fname in goods[:1000]:
    labels[fname] = 1
    src = os.path.join('./', fname)
    dst = os.path.join(train_goods_dir, fname)
    shutil.copyfile(src, dst) 
    
train_bads_dir = os.path.join(train_dir, 'bads')
try:
    os.mkdir(train_bads_dir)
except:
    pass

partition['train'] += bads[:1000]


for fname in bads[:1000]:
    labels[fname] = 0
    src = os.path.join('./', fname)
    dst = os.path.join(train_bads_dir, fname)
    shutil.copyfile(src, dst) 
    
test_dir = os.path.join(base_dir,'test')

try:
    os.mkdir(test_dir)
except:
    pass

test_goods_dir = os.path.join(test_dir, 'goods')
try:
    os.mkdir(test_goods_dir)
except:
    pass


partition['test'] = goods[1000:1500]


for fname in goods[1000:1500]:
    labels[fname] = 1
    src = os.path.join('./', fname)
    dst = os.path.join(test_goods_dir, fname)
    shutil.copyfile(src, dst) 
    
test_bads_dir = os.path.join(test_dir, 'bads')
try:
    os.mkdir(test_bads_dir)
except:
    pass


partition['test'] += bads[1000:1500]


for fname in bads[1000:1500]:
    labels[fname] = 0
    src = os.path.join('./', fname)
    dst = os.path.join(test_bads_dir, fname)
    shutil.copyfile(src, dst) 
    
valid_dir = os.path.join(base_dir,'validate')

try:
    os.mkdir(valid_dir)
except:
    pass

valid_goods_dir = os.path.join(valid_dir, 'goods')
try:
    os.mkdir(valid_goods_dir)
except:
    pass


partition['validate'] = goods[1500:]


for fname in goods[1500:]:
    labels[fname] = 1
    src = os.path.join('./', fname)
    dst = os.path.join(valid_goods_dir, fname)
    shutil.copyfile(src, dst) 
    
valid_bads_dir = os.path.join(valid_dir, 'bads')
try:
    os.mkdir(valid_bads_dir)
except:
    pass

partition['validate'] += bads[1500:]


for fname in bads[1500:]:
    labels[fname] = 0
    src = os.path.join('./', fname)
    dst = os.path.join(valid_bads_dir, fname)
    shutil.copyfile(src, dst) 
    


In [ ]:
from keras.utils import np_utils

class DataGenerator(keras.utils.all_utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(210,210), n_channels=210,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            if self.labels[ID] == 1:
                src = os.path.join(train_goods_dir, str(ID))
            else:
                src = os.path.join(train_bads_dir, str(ID))
            X[i,] = np.load(src)['arr_0']

            # Store class
            y[i] = self.labels[ID]

        return X, y

class ValidateGenerator(keras.utils.all_utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(210,210), n_channels=210,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            if self.labels[ID] == 1:
                src = os.path.join(valid_goods_dir, str(ID))
            else:
                src = os.path.join(valid_bads_dir, str(ID))
            X[i,] = np.load(src)['arr_0']

            # Store class
            y[i] = self.labels[ID]

        return X, y

In [ ]:
training_generator = DataGenerator(partition['train'], labels)
validation_generator = ValidateGenerator(partition['validate'], labels)

In [ ]:
from keras import layers
from keras import models


model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape = (210,210,210)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(8, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
from tensorflow.keras import optimizers
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(learning_rate=1e-4),
             metrics=['acc'])


In [ ]:
history = model.fit(
    training_generator,
    steps_per_epoch=30,
    epochs=20,
    validation_data=validation_generator,
validation_steps=30)